In [3]:
import os
from dotenv import load_dotenv
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.faiss import FAISS
from langchain.schema import Document
import numpy as np
import faiss
import pandas as pd
import openai

In [4]:
# Load environment variables from .env file
load_dotenv()

# Load API key from environment variable
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("The environment variable OPENAI_API_KEY is not set.")

In [5]:
# Set the OpenAI API key
openai.api_key = openai_api_key

In [6]:
# Initialize the OpenAI embeddings
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

# Load your data
df = pd.read_json('new.json')

# Flatten the section-content into a single string for each entry
df['section-content-text'] = df['section-content'].apply(
    lambda x: ' '.join(x.get('Content', [])) if isinstance(x, dict) else ''
)
df = df.drop(columns=['section-content'])

C:\Users\Dilshad\AppData\Local\Temp\ipykernel_37980\1570693540.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)


In [7]:

def find_similar_sections(query, faiss_index, df, top_k=3):
    # Embed the query
    query_embedding = embeddings.embed_query(query)

    # Ensure query_embedding is a 2D numpy array
    query_embedding = np.array([query_embedding], dtype=np.float32)

    # Search the FAISS index for the nearest neighbors
    distances, indices = faiss_index.index.search(query_embedding, top_k)

    # Retrieve the corresponding rows from the DataFrame
    similar_sections = df.iloc[indices[0]]

    return similar_sections

# Test the search with a query


def test_query(query, df, top_k=3):
    # Load the FAISS index
    index = faiss.read_index("faiss_index.index")

    # Create FAISS object with the loaded index
    faiss_index = FAISS(index=index, embedding_function=embeddings.embed_query)

    # Find the top similar sections
    similar_sections = find_similar_sections(query, faiss_index, df, top_k)

    return similar_sections

In [8]:

# Example query
query = "The Chief Justice of the High Court may constitute a Commercial Court."

# Test the query
similar_sections = test_query(query, df, top_k=3)

# Display the results
print(similar_sections[['chapter', 'chapter-title',
      'Section', 'section-content-text']])

TypeError: FAISS.__init__() missing 2 required positional arguments: 'docstore' and 'index_to_docstore_id'

In [11]:
import pandas as pd

# Load your original data
df = pd.read_json('new.json')

# Flatten the section-content into a single string for each entry
df['section-content-text'] = df['section-content'].apply(
    lambda x: ' '.join(x.get('Content', [])) if isinstance(x, dict) else ''
)

# Drop the original 'section-content' column
df = df.drop(columns=['section-content'])

# Save the modified DataFrame to a new JSON file
df.to_json('new_update.json', orient='records', lines=False, indent=4)

print("new_update.json created successfully.")

new_update.json created successfully.
